In [1]:
!pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.7 MB/s eta 0:00:00


In [2]:
from datasets import Dataset
import pandas as pd

# Örnek veri seti (her satır bir şiir veya şiir parçası)
siirler = [
    "Sana gitme demeyeceğim, üşüyorsun ceketimi al",
    "Ben sana mecburum bilemezsin",
    "Bir gece ansızın gelebilirim",
    "Sevda sözleri ettim sana, inadına"
]

df = pd.DataFrame({"text": siirler})
dataset = Dataset.from_pandas(df)


In [3]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained("gpt2")


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [11]:
from datasets import Dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer
import torch

# ---------------------
# 📁 1. VERİ SETİ (Örnek Türkçe şiirler)
# ---------------------
data = {
    "text": [
 "Gökyüzü masmavi bir hüzünle dolar\nKalbimde sensizliğin yankısı var\n",
    "Gece inerken sessizliğe\nYıldızlar düşer hayallere\n",
    "Aşkla dolu bir çiçek gibi\nSoluyorum seni her gece\n",
    "Sensiz geçen her an, bir eksiklik\nKalbim seni bekler gizlice\n",
    "Karanlıkta parlayan umutlar\nGözlerinde hayat bulur\n",
    "Rüzgar savurur yaprakları\nBen seni düşünürüm usul usul\n",
    "Sevda dolu bir şarkı gibi\nÇalar kalbimde her daim\n",
    "Yollar uzun, engeller çok\nAma sen varsın yanımda hep\n",
    "Düşlerimde seninle yürürüm\nSonsuzlukta el ele\n",
    "Bir gülüşün yeter bana\nDünyam aydınlanır o an\n",
    "Kalbim seninle atıyor her saniye\nSensiz hayat anlamsız, yorgun ve sessiz\n",
    "Gözlerin ışık gibi parlıyor karanlıkta\nYalnızlık dağılır, sevda başlar adım adım\n",
    "Rüzgarın fısıldadığı kelimeler senin adın\nHer gece düşlerimde seni çağırırım\n",
    "Deniz dalgaları gibi kıpır kıpır hisler\nKalbim sana köle, sevdanın esiri\n",
    "Yıldızlar kadar parlak hayallerimiz\nGeceyi aydınlatır, sonsuza kadar\n",
    "Aşkın rüzgarında savrulur ruhum\nSensiz geçen günler eksik, yarım\n",
    "Sevdanın sıcaklığı sarar bedenimi\nHer an seninle, kalbim sana ait\n",
    "Gecenin sessizliğinde duyduğum fısıltı\nSenin sesin gibi, huzur veren\n",
    "Bir damla gözyaşında saklı bütün duygular\nSeni sevmenin tarifsiz mutluluğu\n",
    "Gözlerin gözlerimde buluşur sonsuzlukta\nSevginle dolar içim, huzurla dolu\n",
    "Yalnızlık bazen dost olur gecelerde\nAma sen yanımda olunca her şey güzel\n",
    "Karanlık gecede parlayan bir yıldız gibi\nSensin hayatımın en parlak ışığı\n",
    "Her yeni güne seninle başlamak isterim\nAşkınla dolsun her an, her saniye\n",
    "Duygularım coşar, kalbim heyecanla dolar\nSeni düşünmekle geçer her gece\n",
    "Sevgiyle yazılmış bir şiir gibi\nHer mısrada senin adın saklı\n",
    "Birlikte yürürüz sonsuz yollarda\nEl ele, kalp kalbe sevgiyle bağlı\n",
    "Gözlerin deniz, derin ve büyülü\nKendimi kaybederim o mavilikte\n",
    "Rüzgar gibi hafif, özgür ve serin\nSeninle hayat daha güzel, anlamlı\n",
    "Gecenin koynunda saklı umutlar\nSeninle yeşerir, hayat bulur\n",
    "Aşkımızı anlatmak kelimeler kifayetsiz\nAma kalbim hep seninle atar\n",
    "Sevdanın ateşi yanar içimde durmadan\nSeni sevmek en güzel his her zaman\n",
    "Yıldızlar kadar parlak ve sonsuz\nAşkımızın hikayesi yazılıyor\n",
    "Kalbimde bir bahar açar her gün\nSenin sevginle renklenir hayatım\n",
    "Geceyi delen bir ışık gibi\nSeninle hayat aydınlanır, parlar\n",
    "Sevgiyle dolu ellerini tutmak\nDünyanın en güzel huzuru bana\n",
    "Gözlerin gözlerimde kaybolurken\nZaman durur, dünya susar\n",
    "Rüzgarın sesi senin şarkın gibi\nKalbimde yankılanır hiç bitmeden\n",
    "Bir gülüşünle açar çiçekler\nKaranlıklar kaybolur aniden\n",
    "Sevgiyle örülü bir dünya kurmak\nSeninle yaşamak en güzel düşüm\n",
    "Kalbim seni çağırır her saniye\nSonsuzlukta seninle buluşmak\n",
    "Duygularımı anlatmaya kelimeler yetmez\nAma sen bilirsin kalbimin dilini\n",
    "Yalnızlık bazen en yakın arkadaştır\nAma sen yanımdayken mutluluk başkadır\n",
    "Aşkın en güzel haliyle sarar beni\nSeninle tamamlanır her anım\n",
    "Gecenin sessizliğinde seni düşünürüm\nKalbim seninle huzur bulur\n",
    "Rüzgarla savrulan yaprak misali\nSensiz geçen günler anlamsız\n",
    "Sevgiyle dolu kalbim sana ait\nSonsuza dek sürecek bu hikaye\n",
    "Yıldızlar kadar parlak ve güzel\nAşkımızı anlatan şiirler yazılır\n",
    "Bir bakışın yeter bana her zaman\nDünyam seninle tamamlanır\n",
    "Kalbimde saklı en derin sırlar\nSensin hayatımın anlamı\n",
    "Gözlerinle başlayan her gün\nAşkınla dolup taşar içim\n",
    "Rüzgar gibi özgür ve hafif\nSeninle hayat daha güzel\n",
    "Gecenin koynunda saklı umutlar\nSeninle yeşerir, canlanır\n",
    "Aşkımızın sıcaklığı sarar bedenimi\nSensiz hayat anlamsız, boş\n",
    "Sevgiyle yazılmış bir şiir gibi\nHer dizede senin adın geçer\n",
    "Birlikte yürürüz sonsuz yollarda\nEl ele, kalp kalbe bağlı\n",
    "Gözlerin deniz, derin ve büyülü\nKaybolurum o mavilikte\n",
    "Rüzgarın fısıldadığı kelimeler\nSeni anlatır sessizce\n",
    "Sevdanın ateşi yanar içimde\nSeni sevmek en güzel his\n",
    "Yıldızlar kadar parlak ve sonsuz\nAşkımızın hikayesi yaşanır\n",
    "Kalbimde bahar açar her an\nSenin sevginle dolup taşar\n",
    "Geceyi delen bir ışık gibi\nSeninle hayat aydınlanır\n",
    "Sevgiyle dolu ellerini tutmak\nDünyanın en güzel huzuru\n",
    "Gözlerin gözlerimde kaybolur\nZaman durur, dünya susar\n",
    "Rüzgarın sesi senin şarkın\nKalbimde yankılanır sürekli\n",
    "Bir gülüşünle açar çiçekler\nKaranlıklar aniden kaybolur\n",
    "Sevgiyle örülü bir dünya kurmak\nSeninle yaşamak en güzel düş\n",
    "Kalbim seni çağırır her an\nSonsuzlukta seninle buluşmak\n",
    "Duygularımı anlatmaya kelimeler yetmez\nAma sen bilirsin kalbimin dilini\n",
    "Yalnızlık bazen en yakın dost\nAma sen yanımda mutluluk başkadır\n",
    "Aşkın en güzel haliyle sarar beni\nSeninle tamamlanır her an\n",
    "Gecenin sessizliğinde seni düşünürüm\nKalbim seninle huzur bulur\n",
    "Rüzgarla savrulan yaprak misali\nSensiz geçen günler anlamsız\n",
    "Sevgiyle dolu kalbim sana ait\nSonsuza dek sürecek bu hikaye\n",
    "Yıldızlar kadar parlak ve güzel\nAşkımızı anlatan şiirler yazılır\n",
    "Bir bakışın yeter bana her zaman\nDünyam seninle tamamlanır\n",
    "Kalbimde saklı en derin sırlar\nSensin hayatımın anlamı\n",
    "Gözlerinle başlayan her gün\nAşkınla dolup taşar içim\n",
    "Rüzgar gibi özgür ve hafif\nSeninle hayat daha güzel\n",
    "Gecenin koynunda saklı umutlar\nSeninle yeşerir, canlanır\n",
    "Aşkımızın sıcaklığı sarar bedenimi\nSensiz hayat anlamsız, boş\n",
    "Sevgiyle yazılmış bir şiir gibi\nHer dizede senin adın geçer\n",
    "Birlikte yürürüz sonsuz yollarda\nEl ele, kalp kalbe bağlı\n",
    "Gözlerin deniz, derin ve büyülü\nKaybolurum o mavilikte\n",
    "Rüzgarın fısıldadığı kelimeler\nSeni anlatır sessizce\n",
    "Sevdanın ateşi yanar içimde\nSeni sevmek en güzel his\n",
    "Yıldızlar kadar parlak ve sonsuz\nAşkımızın hikayesi yaşanır\n",
    "Kalbimde bahar açar her an\nSenin sevginle dolup taşar\n",
    "Geceyi delen bir ışık gibi\nSeninle hayat aydınlanır\n",
    "Sevgiyle dolu ellerini tutmak\nDünyanın en güzel huzuru\n",
    "Gözlerin gözlerimde kaybolur\nZaman durur, dünya susar\n",
    "Rüzgarın sesi senin şarkın\nKalbimde yankılanır sürekli\n",
    "Bir gülüşünle açar çiçekler\nKaranlıklar aniden kaybolur\n",
    "Sevgiyle örülü bir dünya kurmak\nSeninle yaşamak en güzel düş\n",
    ]
}

dataset = Dataset.from_dict(data)

# ---------------------
# 🔧 2. Tokenizer ve Model
# ---------------------
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # GPT-2'nin pad_token'ı yoktur

model = GPT2LMHeadModel.from_pretrained(model_name)

# ---------------------
# 🧼 3. Tokenizasyon Fonksiyonu
# ---------------------
def tokenize_function(example):
    result = tokenizer(
        example["text"],
        truncation=True,
        max_length=128,
        padding="max_length"
    )
    result["labels"] = result["input_ids"].copy()
    return result

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# ---------------------
# ⚙️ 4. Eğitim Ayarları
# ---------------------
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    num_train_epochs=5,
    logging_steps=10,
    save_steps=100,
    save_total_limit=1,
    report_to="none",  # wandb kapalı
    evaluation_strategy="no"
)

# ---------------------
# 🧪 5. Trainer ve Eğitim
# ---------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

trainer.train()


Map:   0%|          | 0/93 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-11-73aea86d626c>:149: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
10,2.341500
20,1.261700
30,1.099000
40,0.946300
50,0.915500
60,0.759200
70,0.748100
80,0.672800
90,0.685700
100,0.576600


TrainOutput(global_step=235, training_loss=0.6925263628046563, metrics={'train_runtime': 60.5361, 'train_samples_per_second': 7.681, 'train_steps_per_second': 3.882, 'total_flos': 30375198720000.0, 'train_loss': 0.6925263628046563, 'epoch': 5.0})

In [12]:
def generate_poem(prompt="Aşk nedir", max_length=100):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
    attention_mask = torch.ones(input_ids.shape, dtype=torch.long).to(model.device)

    model.eval()
    with torch.no_grad():
        output = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,  # 💡 Bu satır kritik
            max_length=max_length,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(output[0], skip_special_tokens=True)

print(generate_poem("Yalnızlık"))


Yalnızlık kalbim seninle tamamlanır
Senin dünyam seni kelimeler

